In [35]:
import numpy as np
import matplotlib.pyplot as plt

In [36]:
# Initialization of Constants
P_thruster = 1250000 #newtons
P_lift = 1400000 #newtons
P_weight = 500000 #newtons

alpha = 5 #degrees
alpha = alpha * np.pi/180
EC_11 = 148.24e9 #Pascals, AS4 Axial Young's Modulus
EC_22 = 10.07e9 #Pascals, AS4 Transverse Young's Modulus
vC_12 = 0.3 # Major in-plane Poisson's ratio
vC_21 = EC_22*vC_12/EC_11 # Minor in-plane Poisson's ratio
GC_12 = 5.58e9 #Gigapascals, Shear Modulus 

EAl = 69.69e9 #Pascals

R_c = 1.5
R_a = 1.495
R_i = 1.49



In [41]:
# Determining Axial Stiffness, S
def getE(beta):
    beta = beta * np.pi/180 #converting degrees to radians
    
    # Reduced Stiffness Matrix in Local Coordinates
    Q = np.array([[(EC_11)/(1-vC_12*vC_21), (vC_21*EC_11)/(1-vC_12*vC_21), 0], 
                  [(vC_12*EC_22)/(1-vC_12*vC_21), (EC_22)/(1-vC_12*vC_21), 0],
                  [0, 0, GC_12]])

    # Reuter's Matrix
    R = np.array([[1,0,0],[0,1,0],[0,0,2]]) 
    
    # Rotation Matrix
    T = np.array([[np.cos(beta)**2, np.sin(beta)**2, 2*np.sin(beta)*np.cos(beta)],
                 [np.sin(beta)**2, np.cos(beta)**2,-2*np.sin(beta)*np.cos(beta)],
                 [-1*np.sin(beta)*np.cos(beta), np.sin(beta)*np.cos(beta),np.cos(beta)**2 - np.sin(beta)**2]])
    
    # Reduced Stiffness Matrix in Global Coordinates
    Q_bar = np.matmul(np.linalg.inv(T), np.matmul(Q, np.matmul(R, np.matmul(T,np.linalg.inv(R)))))

    S_bar = np.linalg.inv(Q_bar)

    E_axial = 1/S_bar[0,0]
    return E_axial

E_layers = np.array([getE(0), getE(45), getE(-45), getE(90), getE(90), getE(-45), getE(45), getE(0)]) 

# Equations for P1 & P2 (Horizontal and Vertical Resultant Forces)
P1 = P_lift*np.sin(alpha) - P_thruster
P2 = P_lift*np.cos(alpha) - P_weight*np.cos(alpha)

# Calculates S for each layer and sums them for total S
# iterate from the outer radius in
r = R_c
S = 0
i = 0
numLayers = 8
while i < numLayers:
    S = S + (E_layers[i]) * np.pi * (r**2 - (r - 0.005/8)**2)
    r = r - 0.005/8
    i = i + 1
S = S + np.pi * EAl * (R_a**2 - R_i**2)
print("Value of S for laminate is:", S, "N/m")

# Axial Displacement
x1 = np.linspace(0,20,50)
u1 = (P_lift*np.sin(alpha)*x1 - P_thruster*x1)/S

# Axial Strain
eps1 = (P_lift*np.sin(alpha) - P_thruster)/S

# Axial Stress
stress_layers = E_layers * eps1 
print(stress_layers)



Value of S for laminate is: 5464167650.135862 N/m
[-30601558.45858142  -2936298.26529841  -2936298.26529841
  -2078775.59145922  -2078775.59145922  -2936298.26529841
  -2936298.26529841 -30601558.45858142]
